# Image Classification using CNN (FastAI)
Recreated after Stage 1 completion.

In [ ]:
# 1. Environment setup
!pip install -q fastai==2.7.12 fastcore==1.5.27
import fastai, torch
print('fastai', fastai.__version__, 'torch', torch.__version__)

In [ ]:
# 2. GPU check
import torch
print('CUDA available:', torch.cuda.is_available())
print('Device:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')

In [ ]:
# 3. Folder paths
import os
BASE = '/content/ML-Projects/Project-01-CNN'
DATA_DIR = os.path.join(BASE, 'data_sample')
RESULTS_DIR = os.path.join(BASE, 'results')
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
print('DATA_DIR:', DATA_DIR)
print('RESULTS_DIR:', RESULTS_DIR)

In [ ]:
# 4. FastAI DataBlock
from fastai.vision.all import *
def get_dls(img_size=224, bs=64):
    db = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        splitter=RandomSplitter(valid_pct=0.2, seed=42),
        get_y=parent_label,
        item_tfms=Resize(img_size),
        batch_tfms=aug_transforms(mult=1.0))
    return db.dataloaders(os.path.join(DATA_DIR,'seg_train'), bs=bs)
dls = get_dls()
dls.show_batch(max_n=9, figsize=(6,6))

In [ ]:
# 5. Baseline ResNet34 training
learn = cnn_learner(dls, resnet34, metrics=accuracy)
lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
learn.fine_tune(5, base_lr=lr_steep)
learn.save(os.path.join(RESULTS_DIR, 'fastai_resnet34_baseline'))
learn.export(os.path.join(RESULTS_DIR, 'exported_fastai_resnet34.pkl'))

In [ ]:
# 6. Evaluate
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(6,6))
acc = learn.validate()[1].item()
import json; json.dump({'accuracy': acc}, open(os.path.join(RESULTS_DIR, 'metrics.json'),'w'))
print('Saved metrics.json with accuracy =', acc)